### Tensor shape 변경
- reshape(), view() : 원소 개수가 유지됨!, 기존 텐서 공유함

In [2]:
# 모듈 로딩
import torch

In [3]:
# 텐서 데이터 생성
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(t1.shape, t1.ndim)

torch.Size([2, 3]) 2


In [4]:
print(t1, t1.data_ptr(), t1[0].data_ptr(), t1[1].data_ptr())

tensor([[1, 2, 3],
        [4, 5, 6]]) 3809040404736 3809040404736 3809040404760


In [5]:
# 메모리 저장 위치는 그대로.
print(t1.reshape(1, 6).data_ptr())
print(t1.reshape(3, 2).data_ptr())
print(t1.reshape(6, 1).data_ptr())

3809040404736
3809040404736
3809040404736


In [6]:
# 2x3 -> 3x2 형태 변경 : 원소 6개 동일
# 메모리에서 한번에 읽어들이는 개수(단위) 변화 (메모리에서의 저장 순서는 동일함, Endian 동일)
t1.view(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [7]:
# 2x3 -> 6x1 형태 변경 : 원소 6개 동일
t1.view(6, 1)

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

In [8]:
# 2x3 -> 6x? 형태 변경 : 원소 6개 동일
# ? 는 컴파일러가 알맞게 지정하라는 뜻
t1.view(6, -1)

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

In [9]:
# 2x3 -> ?x1 형태 변경 : 원소 6개 동일
# ? 는 컴파일러가 알맞게 지정하라는 뜻
t1.view(-1, 1)

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

In [10]:
# tensor.reshape()
# view() 보다는 reshape() 사용을 권장. (메모리 상에서 새로운 위치에 만듦, 경우에 따라 deep copy)
# Transpose 하면 원소의 순서가 달라지므로 deep copy.
t1.reshape(6, 1)

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

In [11]:
# 원소 수가 유지되어야 하는데 그렇지 않으므로 오류 발생
# t1.reshape(-1, 7) # 실제 원소는 6개인데, 7개를 표시하라니 오류 발생

전치 : 열 <-> 형 변경

In [12]:
print(t1.shape)

t2 = t1.T
print(t2.shape)

torch.Size([2, 3])
torch.Size([3, 2])


In [13]:
# RuntimeError: view size is not compatible with input tensor's size and stride
# t2.view(-1, 6)

In [14]:
t2.reshape(-1, 6)

tensor([[1, 4, 2, 5, 3, 6]])

In [15]:
t3 = t2.reshape(-1, 6)
print(t3, t3.is_contiguous())

tensor([[1, 4, 2, 5, 3, 6]]) True


### 텐서 데이터의 메머리 저장 정보 즉 메타데이터
- 현재 저장 형태, 검색 방향 정보, 시작 정보

In [16]:
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(t1.shape, t1.ndim)
print(f't1.storage() => {t1.storage()}')
print(f't1.storage_offset() => {t1.storage_offset()}')
print(f't1.stride() => {t1.stride()}')  # (행이 바뀔 때 이동해야 하는 데이터 수, 열이 바뀔 때 이동해야 하는 데이터 수)
print(f't1.is_contiguous() => {t1.is_contiguous()}')

torch.Size([2, 3]) 2
t1.storage() =>  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
t1.storage_offset() => 0
t1.stride() => (3, 1)
t1.is_contiguous() => True


C:\Users\kdp\AppData\Local\Temp\ipykernel_28416\2510581992.py:3: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(f't1.storage() => {t1.storage()}')


In [17]:
t2 = t1.view(-1, 2)
print(t2.shape, t2.ndim)
print(f't2.storage() => {t2.storage()}')
print(f't2.storage_offset() => {t2.storage_offset()}')
print(f't2.stride() => {t2.stride()}')
print(f't2.is_contiguous() => {t2.is_contiguous()}')

torch.Size([3, 2]) 2
t2.storage() =>  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
t2.storage_offset() => 0
t2.stride() => (2, 1)
t2.is_contiguous() => True


### 차원 제거 / 추가
- tensor.squeeze() : 텐서에서 차원이 1인 것 제거
- tensor.unsqueeze(dim) : 텐서에 차원 1인 것 추가

In [18]:
# 데이터 생성
t1 = torch.tensor([[1, 2], [3, 4]])
t2 = torch.tensor([[1, 2, 3, 4]])
t3 = torch.tensor([[[1, 2, 3, 4]]])

In [19]:
print(f't1 => {t1.shape}, {t1.ndim}D, {t1.data_ptr()}')
print(f't2 => {t2.shape}, {t2.ndim}D, {t2.data_ptr()}')
print(f't3 => {t3.shape}, {t3.ndim}D, {t3.data_ptr()}')

t1 => torch.Size([2, 2]), 2D, 3809040465920
t2 => torch.Size([1, 4]), 2D, 3809040466048
t3 => torch.Size([1, 1, 4]), 3D, 3809040466112


In [20]:
t11 = t1.squeeze()
t22 = t2.squeeze()
t33 = t3.squeeze()
print(f't1 차원 축소 -> {t11.shape}, {t11.ndim}D, {t11.data_ptr()}')  # 차원이 1인 것이 없어서 줄지 않음
print(f't2 차원 축소 -> {t22.shape}, {t22.ndim}D, {t22.data_ptr()}')
print(f't3 차원 축소 -> {t33.shape}, {t33.ndim}D, {t33.data_ptr()}')

t1 차원 축소 -> torch.Size([2, 2]), 2D, 3809040465920
t2 차원 축소 -> torch.Size([4]), 1D, 3809040466048
t3 차원 축소 -> torch.Size([4]), 1D, 3809040466112


In [21]:
t34 = t3.squeeze(dim=2)     # 차원 1일 경우 없앨 차원의 지정
print(f't3 차원 축소 -> {t34.shape}, {t34.ndim}D, {t34.data_ptr()}')    # 2번째 차원이 4라서 squeeze 하지 않음

t3 차원 축소 -> torch.Size([1, 1, 4]), 3D, 3809040466112


In [22]:
## 원소/요소 수 변경 없이 1 차원 증가시키가 => torch.unsqueeze(dim)
print(f't1 정보 => {t11.shape}, {t11.ndim}D, {t11.data_ptr()}')

t1 정보 => torch.Size([2, 2]), 2D, 3809040465920


In [23]:
t11 = t1.unsqueeze(dim=0)
t22 = t1.unsqueeze(dim=-1)

print(f't1 차원 추가 => {t11.shape}, {t11.ndim}D, {t11.data_ptr()}, {t11.stride()}')
print(f't1 차원 추가 => {t22.shape}, {t22.ndim}D, {t22.data_ptr()}, {t22.stride()}')

t1 차원 추가 => torch.Size([1, 2, 2]), 3D, 3809040465920, (4, 2, 1)
t1 차원 추가 => torch.Size([2, 2, 1]), 3D, 3809040465920, (2, 1, 1)


In [24]:
t1[None].shape  # 길이가 1인 차원을 추가. (unsqueeze와 동일)

torch.Size([1, 2, 2])

In [25]:
t1.shape

torch.Size([2, 2])